In [18]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform


In [19]:
import pandas as pd
data= pd.read_csv("/home/sagar24/nlp_dataset/emojify/emojify_data.csv",usecols=[0,1],names=['colA', 'colB'])

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

text=data["colA"].tolist()
emojis=np.array(data["colB"])
x_train, x_test, y_train, y_test = train_test_split(text, to_categorical(emojis), test_size=0.2)

print("{} +++> {}".format(text[1],emojis[1]))
maxLen = len(max(text, key=len).split())

print(maxLen)
print(len(x_train))
print(len(x_test))
print(y_train.shape)
print(y_test.shape)

x_train=np.array(x_train)
x_test=np.array(x_test)

work is horrible +++> 3
10
146
37
(146, 5)
(37, 5)


In [20]:
import emoji
emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

In [21]:
index = 5
print(text[index], label_to_emoji(emojis[index]))

what is your favorite baseball game ⚾


In [22]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [23]:
word_to_index, index_to_word, word_to_vec_map=read_glove_vecs("/home/sagar24/nlp codes/8542_11957_compressed_glove.6B.50d.txt/glove.6B.50d.txt")

In [24]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [25]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j + 1
            
    ### END CODE HERE ###
    
    return X_indices

In [26]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [27]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer =  pretrained_embedding_layer(word_to_vec_map, word_to_index)
    print(embedding_layer.shape)
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(rate =  0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(rate =  0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(units = 5, activation = 'softmax')(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices,X)
    
    ### END CODE HERE ###
    
    return model

In [28]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

AttributeError: 'Embedding' object has no attribute 'shape'

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
X_train_indices = sentences_to_indices(x_train, word_to_index, maxLen)


In [15]:
model.fit(X_train_indices, y_train, epochs = 70, batch_size = 32, shuffle=True)

Epoch 1/70
146/146 [==============================] - 1s 10ms/step - loss: 1.6009 - accuracy: 0.2808
Epoch 2/70
146/146 [==============================] - 0s 692us/step - loss: 1.5684 - accuracy: 0.3219
Epoch 3/70
146/146 [==============================] - 0s 718us/step - loss: 1.5411 - accuracy: 0.4110
Epoch 4/70
146/146 [==============================] - 0s 786us/step - loss: 1.5214 - accuracy: 0.3973
Epoch 5/70
146/146 [==============================] - 0s 716us/step - loss: 1.4946 - accuracy: 0.4041
Epoch 6/70
146/146 [==============================] - 0s 751us/step - loss: 1.4766 - accuracy: 0.4521
Epoch 7/70
146/146 [==============================] - 0s 780us/step - loss: 1.4505 - accuracy: 0.4589
Epoch 8/70
146/146 [==============================] - 0s 805us/step - loss: 1.4172 - accuracy: 0.5000
Epoch 9/70
146/146 [==============================] - 0s 768us/step - loss: 1.4176 - accuracy: 0.4726
Epoch 10/70
146/146 [==============================] - 0s 741us/step - loss: 1.3654

In [17]:
X_test_indices = sentences_to_indices(x_test, word_to_index, max_len = maxLen)

loss, acc = model.evaluate(X_test_indices, y_test)
print("Loss = ",loss)
print("Test accuracy = ", acc)

37/37 [==============================] - 0s 405us/step
Loss =  1.3116702131322913
Test accuracy =  0.5675675868988037
